## Building SQL Agents using LLM that will transform the natural language query into SQL query and interact with the database.

In [7]:
# Load the csv data into sqlite database and test run the sql query
# Load data from csv
import pandas as pd
data = pd.read_csv('/Users/vss/Work/Learnings_Sourced_v_Not_HighScoring/input/Source Data 02-21-24.csv', encoding='latin1')
print(data.shape)

(5644, 92)


In [8]:
# Define Language model
from langchain_ollama import OllamaLLM
llm_model = OllamaLLM(model="deepseek-r1:7b")#'llama3.1:8b')#") # Language model for answer generation
#llm_model = OllamaLLM(model="deepseek-r1:7b") # Language model for answe generation
llm_model.invoke("Hi")

'<think>\n\n</think>\n\nHello! How can I assist you today? 😊'

In [33]:
question = "Extract all deals specifically associated with the company 'Imperfect Foods' from the given list of deals."
context = ', '.join(data['Opportunity Name'])
#context = 'Imperfect Foods - Seed, Meta - Series A, Imperfect Foods - Series B, Fundly - SAFE'
#print(context)
#print(data['Opportunity Name'].str.contains('Imperfect Foods'))
context = ', '.join(data.loc[data['Opportunity Name'].str.contains('Imperfect Foods'), 'Opportunity Name'])
print(', '.join(data.loc[data['Opportunity Name'].str.contains('Imperfect Foods'), 'Opportunity Name']))

Imperfect Foods (fka Imperfect Produce) Series Seed, Imperfect Foods (fka Imperfect Produce) Series A, Imperfect Foods (fka Imperfect Produce) Bridge, Imperfect Foods (fka Produce) Series B / Secondary, Imperfect Foods Bridge Note, Imperfect Foods Bridge Note, Imperfect Foods Series C follow on


In [34]:
# The above chain invokation may not give the correct SQL query. So, we can use the following code to get the correct SQL query.

from langchain_core.prompts import ChatPromptTemplate
PROMPT_TEMPLATE = """
You are an expert assistant. Use the provided context to answer the query.

Query: "{user_question}"

Context: "{context}"
"""
prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
response_chain = prompt | llm_model
response = response_chain.invoke({"user_question": question, "context": context})
print(response)


<think>
Okay, so I need to figure out how to extract all the deals related to "Imperfect Foods" from the given list. Let me start by understanding what's provided.

The context is a series of terms listed: "Imperfect Foods (fka Imperfect Produce) Series Seed," then another one with "A," followed by "B / Secondary." Then some notes marked as "Bridge" and another "Series C follow on."

I think the goal here is to identify which of these are actual deals. The context mentions that each deal has a type, like Series, A, B, etc., and some have notes or are follow-ups.

So, I should list all items starting with "Imperfect Foods" and then note their types. The "Series Seed," "A," "B / Secondary" sound like different stages or types of deals. The terms after the slash might indicate additional information, so "Bridge Note" could be a type of deal as well.

I'll need to extract each unique term that starts with "Imperfect Foods" followed by its suffixes and any modifiers.
</think>

The deals rel

In [ ]:
# Create a sqlite database and load the data
from sqlalchemy import create_engine
import os
if not os.path.exists('cv_data.db'):
    engine = create_engine('sqlite:///cv_data.db', echo=False)
    data.to_sql('sourced_data', engine, index=False)
    print("A .db (kind of txt) file is created in the current directory.")
else:
    print("The .db file already exists in the current directory.")


In [ ]:
# Test run the sql query
from langchain_community.utilities import  SQLDatabase

db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT count(*) from sourced_data where `CV Contact Type`='Portfolio Company';")


In [ ]:
db.get_usable_table_names()
db.table_info

In [ ]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm_model, db)
chain.get_prompts()[0].pretty_print()
print(chain)

In [ ]:
question = "What is the distribution of `CV Contact Type`"
question = "How many deals are there in the sourced_data table"
question = "How many deals are there in the sourced_data table that is closed by correlation"
query_first_attempt = chain.invoke({"question": question})
print(query_first_attempt)

In [ ]:
# The above chain invokation may not give the correct SQL query. So, we can use the following code to get the correct SQL query.

from langchain_core.prompts import ChatPromptTemplate
PROMPT_TEMPLATE = """
You are an expert SQL query creator. Build a SQL query to answer the following user question using the provided context and few rows from the table.
Only return SQL quesry in string format.

Question: {user_question}
Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
response_chain = prompt | llm_model
response = response_chain.invoke({"user_question": question, "context": query_first_attempt})
print(response)


In [ ]:
db.run("""SELECT * FROM sourced_data limit 10;""")

In [ ]:
from langchain_experimental.agents import create_pandas_dataframe_agent

agent_executor = create_pandas_dataframe_agent(
    llm_model,
    data,
    verbose=True,
    allow_dangerous_code=True,
    include_df_in_prompt=False
)
print(agent_executor)
question1 = "What is the distribution of `CV Contact Type`" # Ran Successfull
question2 = "How many deals that are sourced after 2020" # Failed, becasue date_sourced could not found
question3 = "How many deals that are sourced after 2020? use `Date Docs Received` as date sourced." # Failed, becasue date_sourced could not found
agent_executor.invoke({"input": question3})

In [ ]:
llm_model

In [ ]:
# Now create a SQL Agent using LLM that will transform the natural language query into SQL query and interact with the database
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm_model, db=db, agent_type="zero-shot-react-description", verbose=True, handle_parsing_errors=True, max_iterations=30)
print(agent_executor)
agent_executor.invoke({"input": "How many rows are there in the sourced_data table"})